# Using a Vector Database to Recommend Movies

Vector search is certainly critical for generative AI, but also has lots of other interesting applications as well. One very common one is building personalized recommendations. In this exercise, we'll take a small diversion and build a quick movie recommender using a vector database.

For this exercise we'll use the [MovieLens Latest Small Dataset](https://grouplens.org/datasets/movielens/latest/), which contains 100,000 ratings and 3,600 tags applied to 9,000 movies by 600 users. The strategy we'll use is to create embeddings for the movies based on the user ratings. Then if a user rated a particular movie highly, we'll recommend "similar" movies, as determined by the embeddings

In [2]:
import lancedb

import numpy as np
import pandas as pd

The dataset is included along with this exercise:

In [2]:
!ls ml-latest-small

README.txt  links.csv  movies.csv  ratings.csv	tags.csv


## Loading data

Let's start by reading in the `ratings.csv` file. We'll use this to compute the content embeddings

In [3]:
ratings = pd.read_csv('./ml-latest-small/ratings.csv', header=0)
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [102]:
ratings.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

## Computing ratings

Use the ratings dataframe from above and create a new reviews dataframe of users (index) and movies (columns). Each entry (i, j) in the dataframe will be the rating that user_i gave to movie_j. If no such pair exists, then fill in the value 0.

**HINT** In Excel this would be called a **pivot** table

In [5]:
reviewmatrix = ratings[['userId','movieId', 'rating']].pivot(index='userId',columns=['movieId'],values='rating').fillna(0)

In [6]:
reviewmatrix

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Computing embeddings

Now let's use [matrix factorization](https://www.cs.cmu.edu/~mgormley/courses/10601-s17/slides/lecture25-mf.pdf) to extract content embeddings.

Please compute the content embeddings from the reviewmatrix dataframe and name the result `embeddings`.

**HINT**
1. SVD is a popular matrix factorization technique
2. If you're not sure which of the SVD results to use as the content embeddings, look at the shape of the results

In [7]:
matrix = reviewmatrix.values


In [8]:
matrix

array([[4. , 0. , 4. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [2.5, 2. , 2. , ..., 0. , 0. , 0. ],
       [3. , 0. , 0. , ..., 0. , 0. , 0. ],
       [5. , 0. , 0. , ..., 0. , 0. , 0. ]])

In [9]:
%%time
# matrix factorization using SVD
# _, _, vh = np.linalg.svd(matrix)

u, s, vh = np.linalg.svd(matrix)
embeddings = vh.T

CPU times: user 1min 14s, sys: 54.6 s, total: 2min 9s
Wall time: 2min 9s


In [10]:
u.shape

(610, 610)

In [14]:
s.shape, vh.shape

((610,), (9724, 9724))

## Metadata

Read in the `movies.csv` and `links.csv` files and make sure it is aligned with the embeddings dataframe.

**HINT** pandas provides `reindex` functionality to help with data alignment

In [15]:
movies = pd.read_csv('./ml-latest-small/movies.csv', header=0)
movies = movies.set_index("movieId").reindex(reviewmatrix.columns)
movies

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
193585,Flint (2017),Drama


In [18]:
len(pd.read_csv('./ml-latest-small/movies.csv', header=0)['movieId'].unique())

9742

In [10]:
# pd.read_csv('./ml-latest-small/movies.csv', header=0)
reviewmatrix.columns

Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
       ...
       601, 602, 603, 604, 605, 606, 607, 608, 609, 610],
      dtype='int64', name='userId', length=610)

In [22]:
# pd.read_csv('./ml-latest-small/links.csv', header=0)

In [21]:
# now do this for links

links = pd.read_csv('./ml-latest-small/links.csv', header=0)
links = links.set_index("movieId").reindex(reviewmatrix.columns)
links

,imdbId,tmdbId
movieId,,
1,114709,862.0
2,113497,8844.0
3,113228,15602.0
4,114885,31357.0
5,113041,11862.0
...,...,...
193581,5476944,432131.0
193583,5914996,445030.0
193585,6397426,479308.0


## Create vector database table

Let's create a table with the following fields:

1. an integer movie id field
2. a vector field of embeddings
3. a string field of genres
4. a string field for the movie title
5. an integer field for the imdb_id

First, we'll create a pydantic model named `Content` for these fields. For the vector field, use the `lancedb.pydantic.vector` as a shorthand for the field type. Note that you'll need to pass in the number of dimensions.

In [23]:
from lancedb.pydantic import vector, LanceModel

class Content(LanceModel):
    movie_id: int
    vector: vector(embeddings.shape[1])
    genres: str
    title: str
    imdb_id: int
        
    @property
    def imdb_url(self) -> str:
        return f"https://www.imdb.com/title/tt{self.imdb_id}"

In [42]:
# dir(Content)

In [34]:
Content.field_names() #schema().keys()

['movie_id', 'vector', 'genres', 'title', 'imdb_id']

Let's prepare a list of python dicts with all of the data

In [77]:
values = list(zip(*[reviewmatrix.columns,
                    embeddings, 
                    movies["genres"], 
                    movies["title"], 
                    links["imdbId"], 
                    links["tmdbId"]]))
keys = Content.field_names()
data = [dict(zip(keys, v)) for v in values]

data[0]

{'movie_id': 1,
 'vector': array([-0.0704499 ,  0.02759119, -0.07844388, ..., -0.00101062,
        -0.00101062,  0.00389649]),
 'genres': 'Adventure|Animation|Children|Comedy|Fantasy',
 'title': 'Toy Story (1995)',
 'imdb_id': 114709}

In [36]:
embeddings.shape

(9724, 9724)

In [37]:
Content.to_arrow_schema()

movie_id: int64 not null
vector: fixed_size_list<item: float>[9724] not null
  child 0, item: float
genres: string not null
title: string not null
imdb_id: int64 not null

Now please connect to the local database at ~/.lancedb
and create the LanceDB table named "movielens_small".

**HINT** you've seen this in a previous exercise

In [78]:
import pyarrow as pa
table_name = "movielens_small"
data = pa.Table.from_pylist(data, schema=Content.to_arrow_schema())

db = lancedb.connect('~/.lancedb')
db.drop_table("movielens_small")
table = db.create_table("movielens_small", schema=Content)
table.add(data)

In [66]:
# dir(table)

In [79]:
dft=table.to_pandas()
dft[dft["title"]=='Moana (2016)']

,movie_id,vector,genres,title,imdb_id
9412,166461,"[-0.0020678842, -0.0090984395, -0.0022586556, ...",Adventure|Animation|Children|Comedy|Fantasy,Moana (2016),3521164


In [80]:
dft["title"].unique()

array(['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)',
       ..., 'Flint (2017)', 'Bungo Stray Dogs: Dead Apple (2018)',
       'Andrew Dice Clay: Dice Rules (1991)'], dtype=object)

## Generating recommendations

Finally we're ready to generate recommendations based on content vector similarity.

For this exercise please fill in the rest of the function to generate recommendations

**HINT** It's easier if you use the pydantic integration to convert results

In [105]:
def get_recommendations(title: str) -> list[(int, str, str)]:
    # First we retrieve the vector for the input title
    print(table.to_lance().to_table(filter=f"title='{title}'")["vector"].to_numpy())
    query_vector = (table.to_lance().to_table(filter=f"title='{title}'")["vector"].to_numpy()[0])
    # Please write the code to search for the 5 most similar titles
    #res_df = table.search(query_vector).limit(5).to_df()
    res_df = table.search(query_vector).limit(5).to_pydantic(Content)
    
    
    # For each result, return the movie_id, title, and imdb_url
    #return res_df[['movie_id', 'title', 'imdb_id']]
    return [(r.movie_id, r.title, r.imdb_url) for r in res_df]

If a user watched the movie titled "Moana (2016)", what should we recommend to the user?

In [106]:
get_recommendations("Moana (2016)")

[array([-0.00206788, -0.00909844, -0.00225866, ...,  0.00163186,
         0.00163186,  0.00046297], dtype=float32)               ]


[(166461, 'Moana (2016)', 'https://www.imdb.com/title/tt3521164'),
 (2283, 'Sheltering Sky, The (1990)', 'https://www.imdb.com/title/tt100594'),
 (941, 'Mark of Zorro, The (1940)', 'https://www.imdb.com/title/tt32762'),
 (2674,
  'Loss of Sexual Innocence, The (1999)',
  'https://www.imdb.com/title/tt126859'),
 (3150, 'War Zone, The (1999)', 'https://www.imdb.com/title/tt141974')]

In [107]:
get_recommendations("Flint (2017)")

[array([-6.4683605e-05, -5.9758622e-04,  8.7109387e-05, ...,
         9.9109250e-01, -8.9074774e-03,  3.0432513e-05], dtype=float32)]


[(193585, 'Flint (2017)', 'https://www.imdb.com/title/tt6397426'),
 (806, 'American Buffalo (1996)', 'https://www.imdb.com/title/tt115530'),
 (1335, 'Blood Beach (1981)', 'https://www.imdb.com/title/tt82083'),
 (1427, 'Turbulence (1997)', 'https://www.imdb.com/title/tt120390'),
 (4675, 'Three Fugitives (1989)', 'https://www.imdb.com/title/tt98471')]

What about "Rogue One: A Star Wars Story (2016)"?

In [108]:
get_recommendations("Rogue One: A Star Wars Story (2016)")

[array([-0.01500093, -0.03267253,  0.00265089, ...,  0.00221272,
         0.00221272,  0.00132124], dtype=float32)               ]


[(166528,
  'Rogue One: A Star Wars Story (2016)',
  'https://www.imdb.com/title/tt3748528'),
 (2307, 'One Tough Cop (1998)', 'https://www.imdb.com/title/tt122642'),
 (1331, 'Audrey Rose (1977)', 'https://www.imdb.com/title/tt75704'),
 (1940, "Gentleman's Agreement (1947)", 'https://www.imdb.com/title/tt39416'),
 (2801,
  'Oscar and Lucinda (a.k.a. Oscar & Lucinda) (1997)',
  'https://www.imdb.com/title/tt119843')]

Do these look reasonable? How would you improve this recommender system?